# Clean SDE Subsidy Data
This script cleans subsidy data downloaded from RVO.nl and compiled in Excel on SDE and SDE+++ subsidies by removing all records that do not have latitude and longitude.


About the raw data:

The subsidy data from RVO.nl was originally download as XLSX files. Upon initial inspection, it was noted that the files contained extra informational columns and rows that needed to be deleted. The information included images and plain text introduction material. The following columns remain:
 - OBJECTID
 - Programma
 - Projectnummer
 - Adres
 - Postal Code
 - Indienronde
 - Realisatiejaar
 - Kwartaal
 - Aavrager
 - Status
 - Vermogen
 - Maximale_subsidie
 - Looptijd
 - Rechtsvorm
 - Provincie
 - Latitude
 - Longitude
 - Categorie
 - Thema
 - label
 - RES_regio
 - Gemeente
 - Plaats
 - x
 - y


Processing and output:

The file was saved as a CSV and used in the script below. The final output is generates a CSV file.

## Before running:
1. Update directories and file names as necessary under CONFIGURATION




In [1]:
import pandas as pd
import os

# -------------------------------
# CONFIGURATION
# -------------------------------
RAW_DATA_DIR = "../raw_data/"
OUTPUT_DIR = "../clean_data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

input_file = os.path.join(RAW_DATA_DIR, "SDE.csv")
output_file = os.path.join(OUTPUT_DIR, "sde_subsidies_clean.csv")

# -------------------------------
# FUNCTIONS
# -------------------------------
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace(".", "_", regex=False)
    )
    return df

def strip_whitespace(df):
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

def clean_data(df):
    df = clean_column_names(df)
    df = strip_whitespace(df)
    df = df.dropna(how='all')  # Drop entirely empty rows
    df = df.drop_duplicates()
    return df

def remove_missing_coordinates(df):
    if 'latitude' in df.columns and 'longitude' in df.columns:
        df = df.dropna(subset=['latitude', 'longitude'])
    else:
        print("⚠️ Columns 'latitude' and/or 'longitude' not found in the dataset.")
    return df

def validate_and_clean_columns(df):
    # projectnummer as string
    if 'projectnummer' in df.columns:
        df['projectnummer'] = df['projectnummer'].astype(str)

    # programma: drop blanks, count types
    if 'programma' in df.columns:
        df = df[df['programma'].notna() & (df['programma'] != "")]
        programma_counts = df['programma'].value_counts()
        print("📊 Programma types found:\n", programma_counts)

    # maximal_subsidie: convert to int euros (remove decimal)
    if 'maximal_subsidie' in df.columns:
        df['maximal_subsidie'] = (
            df['maximal_subsidie']
            .replace(",", ".", regex=True)
            .astype(float)
            .round(0)
            .astype('Int64')
        )

    # realisatiejaar: ensure it's a valid year, no blanks
    if 'realisatiejaar' in df.columns:
        df = df[df['realisatiejaar'].notna()]
        df['realisatiejaar'] = df['realisatiejaar'].astype(str).str.extract(r'(\d{4})')[0]
        df = df[df['realisatiejaar'].notna()]
        df['realisatiejaar'] = df['realisatiejaar'].astype(int)

    # catagorie: make sure it's string
    if 'catagorie' in df.columns:
        df['catagorie'] = df['catagorie'].astype(str)

    return df

# -------------------------------
# PROCESSING
# -------------------------------
df = pd.read_csv(input_file, sep=",")
df = clean_data(df)
df = remove_missing_coordinates(df)
df = validate_and_clean_columns(df)

# -------------------------------
# SUMMARY
# -------------------------------
print(f"✅ Total rows after full cleaning and validation: {len(df)}")

# -------------------------------
# SAVE OUTPUT
# -------------------------------
df.to_csv(output_file, index=False)
print(f"📁 Cleaned file saved to: {output_file}")


📊 Programma types found:
 programma
SDE    131
Name: count, dtype: int64
✅ Total rows after full cleaning and validation: 131
📁 Cleaned file saved to: ../clean_data/sde_subsidies_clean.csv


C:\Users\kemun\AppData\Local\Temp\ipykernel_28224\2723776948.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


## Minimize SDE Data
The following code saves a minimized csv file with the following columns:
    - projectnummer
    - programma
    - maximal_subsidie
    - realisatiejaar
    - catagorie
    - latitude
    - longitude

## Before running:
1. Update directories and file names as necessary under CONFIGURATION

In [2]:
import pandas as pd
import os

# -------------------------------
# CONFIGURATION
# -------------------------------
CLEANED_FILE = "../clean_data/sde_subsidies_clean.csv"
MINIMIZED_DIR = "../minimized_data/"
GEOCODED_DIR = "../geocoded_data/"
OUTPUT_FILENAME = "sde_minimized_geocoded.csv"

# Create output directories if they don't exist
os.makedirs(MINIMIZED_DIR, exist_ok=True)
os.makedirs(GEOCODED_DIR, exist_ok=True)

# -------------------------------
# COLUMNS TO KEEP
# -------------------------------
COLUMNS_TO_KEEP = [
    "projectnummer",
    "programma",
    "realisatiejaar",
    "categorie",
    "maximale_subsidie",
    "latitude",
    "longitude"
]

# -------------------------------
# LOAD & FILTER
# -------------------------------
df = pd.read_csv(CLEANED_FILE)

# Ensure all required columns exist
missing_cols = [col for col in COLUMNS_TO_KEEP if col not in df.columns]
if missing_cols:
    raise ValueError(f"❌ Missing columns in input file: {missing_cols}")

# Filter only the needed columns
df_minimized = df[COLUMNS_TO_KEEP]

# -------------------------------
# SAVE OUTPUTS
# -------------------------------
df_minimized.to_csv(os.path.join(MINIMIZED_DIR, OUTPUT_FILENAME), index=False)
df_minimized.to_csv(os.path.join(GEOCODED_DIR, OUTPUT_FILENAME), index=False)

print(f"✅ Minimized file saved to both:\n📁 {MINIMIZED_DIR}\n📁 {GEOCODED_DIR}")


✅ Minimized file saved to both:
📁 ../minimized_data/
📁 ../geocoded_data/


# Aggregation based on neighbourhood

This code aggregates subsidies occurence by performing a spatial join between geocoded subsidies data (sde_minimized_geocoded.csv) and neighborhood boundaries (Buurtgrenzen_Zwolle.shp). It calculates the total number of applied subsidies within each neighborhood.

In [8]:
import geopandas as gpd
import pandas as pd
import os

GEOCODED_DATA_DIR = "../geocoded_data/"
NEIGHBORHOOD_DATA_DIR = "../raw_data/"
OUTPUT_DIR = "../aggregated_data/"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define paths explicitly
input_file = os.path.join(GEOCODED_DATA_DIR, "sde_minimized_geocoded.csv")
neighborhood_file = os.path.join(NEIGHBORHOOD_DATA_DIR, "Buurtgrenzen_Zwolle.shp")

# Load SDE subsidy data and convert to GeoDataFrame
subsidy_df = pd.read_csv(input_file)
subsidy_gdf = gpd.GeoDataFrame(
    subsidy_df,
    geometry=gpd.points_from_xy(subsidy_df.longitude, subsidy_df.latitude),
    crs='EPSG:4326'
)

# Load neighborhoods shapefile with correct CRS (EPSG:28992)
neighborhoods_gdf = gpd.read_file(neighborhood_file)

# Convert subsidy points to match neighborhoods CRS (EPSG:28992)
subsidy_gdf = subsidy_gdf.to_crs(neighborhoods_gdf.crs)

# Spatial join: assign subsidies to neighborhoods
joined_gdf = gpd.sjoin(subsidy_gdf, neighborhoods_gdf, predicate='within')

# Count subsidies and sum subsidy amounts per neighborhood
aggregated_subsidies = joined_gdf.groupby('OFFICIËLE').agg(
    subsidy_count=('maximale_subsidie', 'size'),
    total_subsidy_amount=('maximale_subsidie', 'sum')
).reset_index()

# Merge aggregation results back with neighborhood geometry
result_gdf = neighborhoods_gdf.merge(aggregated_subsidies, on='OFFICIËLE', how='left')
result_gdf['subsidy_count'] = result_gdf['subsidy_count'].fillna(0).astype(int)
result_gdf['total_subsidy_amount'] = result_gdf['total_subsidy_amount'].fillna(0)

# Save aggregated data to shapefile
output_path = os.path.join(OUTPUT_DIR, "Zwolle_Neighbourhood_Aggregated_SDESubsidy.shp")
result_gdf.to_file(output_path)

print(f"✅ Aggregated shapefile saved successfully to: {output_path}")


✅ Aggregated shapefile saved successfully to: ../aggregated_data/Zwolle_Neighbourhood_Aggregated_SDESubsidy.shp


C:\Users\kemun\AppData\Local\Temp\ipykernel_28224\147011612.py:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result_gdf.to_file(output_path)
c:\Users\kemun\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'subsidy_count' to 'subsidy_co'
  ogr_write(
c:\Users\kemun\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'total_subsidy_amount' to 'total_subs'
  ogr_write(


# Aggregation based on district

This code aggregates subsidies occurence by performing a spatial join between geocoded subsidies data (sde_minimized_geocoded.csv) and district boundaries (Wijkgrenzen_Zwolle.shp). It calculates the total number of applied subsidies within each district.

In [9]:
import geopandas as gpd
import pandas as pd
import os

GEOCODED_DATA_DIR = "../geocoded_data/"
DISTRICT_DATA_DIR = "../raw_data/"
OUTPUT_DIR = "../aggregated_data/"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define paths explicitly
input_file = os.path.join(GEOCODED_DATA_DIR, "sde_minimized_geocoded.csv")
district_file = os.path.join(DISTRICT_DATA_DIR, "Wijkgrenzen_Zwolle.shp")

# Load SDE subsidy data and convert to GeoDataFrame
subsidy_df = pd.read_csv(input_file)
subsidy_gdf = gpd.GeoDataFrame(
    subsidy_df,
    geometry=gpd.points_from_xy(subsidy_df.longitude, subsidy_df.latitude),
    crs='EPSG:4326'
)

# Load districts shapefile with correct CRS (EPSG:28992)
districts_gdf = gpd.read_file(district_file)

# Convert subsidy points to match districts CRS (EPSG:28992)
subsidy_gdf = subsidy_gdf.to_crs(districts_gdf.crs)

# Spatial join: assign subsidies to districts
joined_gdf = gpd.sjoin(subsidy_gdf, districts_gdf, predicate='within')

# Count subsidies and sum subsidy amounts per district
aggregated_subsidies = joined_gdf.groupby('OFFICIËLE').agg(
    subsidy_count=('maximale_subsidie', 'size'),
    total_subsidy_amount=('maximale_subsidie', 'sum')
).reset_index()

# Merge aggregation results back with district geometry
result_gdf = districts_gdf.merge(aggregated_subsidies, on='OFFICIËLE', how='left')
result_gdf['subsidy_count'] = result_gdf['subsidy_count'].fillna(0).astype(int)
result_gdf['total_subsidy_amount'] = result_gdf['total_subsidy_amount'].fillna(0)

# Save aggregated data to shapefile
output_path = os.path.join(OUTPUT_DIR, "Zwolle_Districts_Aggregated_SDESubsidy.shp")
result_gdf.to_file(output_path)

print(f"✅ Aggregated shapefile saved successfully to: {output_path}")


✅ Aggregated shapefile saved successfully to: ../aggregated_data/Zwolle_Districts_Aggregated_SDESubsidy.shp


C:\Users\kemun\AppData\Local\Temp\ipykernel_28224\3394971484.py:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result_gdf.to_file(output_path)
c:\Users\kemun\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'subsidy_count' to 'subsidy_co'
  ogr_write(
c:\Users\kemun\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'total_subsidy_amount' to 'total_subs'
  ogr_write(
